In [9]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from config import geoapify_key

In [10]:
# Load CSV file created in WeatherPy
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,-10.57,96,75,3.51,RU,1698198233
1,1,ciudad lazaro cardenas,17.9583,-102.2000,25.91,82,100,2.58,MX,1698198233
2,2,hawaiian paradise park,19.5933,-154.9731,28.73,85,75,3.60,US,1698198234
3,3,port lincoln,-34.7333,135.8667,13.74,40,81,9.93,AU,1698197986
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.16,88,41,10.03,SH,1698198234


In [11]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [12]:
# Narrow down cities that fit criteria 

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,al burayqah,30.4062,19.5739,23.87,53,0,2.11,LY,1698198241
46,46,rosetta,31.4044,30.4164,23.33,73,0,2.91,EG,1698198243
94,94,saint george,37.1041,-113.5841,22.33,23,0,3.60,US,1698198212
134,134,as sulayyil,20.4607,45.5779,24.14,51,0,3.75,SA,1698198264
153,153,tsiombe,-25.3000,45.4833,21.37,91,0,2.27,MG,1698198268
161,161,laguna,38.4210,-121.4238,23.16,63,0,2.06,US,1698198270
188,188,morondava,-20.2833,44.2833,25.58,83,0,1.66,MG,1698198276
328,328,les avirons,-21.2418,55.3394,22.43,73,0,1.54,RE,1698198309
330,330,ceres,37.5949,-120.9577,22.97,58,0,3.60,US,1698198276
354,354,tsiroanomandidy,-18.7667,46.0333,22.15,76,0,1.11,MG,1698198315


In [6]:
# Figure out hotel arrangements

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,port hedland,AU,-20.3167,118.5667,88,
111,tindouf,DZ,27.6711,-8.1474,29,
140,fort bragg,US,35.1390,-79.0060,34,
168,laguna,US,38.4210,-121.4238,65,
227,ivins,US,37.1686,-113.6794,24,
245,beira,MZ,-19.8436,34.8389,86,
250,toliara,MG,-23.3500,43.6667,84,
262,mount isa,AU,-20.7333,139.5000,27,
263,greenville,US,35.6127,-77.3663,52,
266,salinas,US,36.6777,-121.6555,73,


In [13]:
# Find the first hotel located within 10,000 metres 

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # JSONify
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotel_df

Starting hotel search
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


,City,Country,Lat,Lng,Humidity,Hotel Name
1,port hedland,AU,-20.3167,118.5667,88,The Esplanade Hotel
111,tindouf,DZ,27.6711,-8.1474,29,محمد بوسبي
140,fort bragg,US,35.1390,-79.0060,34,Airborne Inn Lodging
168,laguna,US,38.4210,-121.4238,65,Holiday Inn Express & Suites
227,ivins,US,37.1686,-113.6794,24,Crescent Moon Inn
245,beira,MZ,-19.8436,34.8389,86,Hotel Savoy
250,toliara,MG,-23.3500,43.6667,84,Ambary
262,mount isa,AU,-20.7333,139.5000,27,Ibis Styles
263,greenville,US,35.6127,-77.3663,52,Quality Inn
266,salinas,US,36.6777,-121.6555,73,The Traveler's Hotel


In [14]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)